# Export Top Texts for SAE Concepts

This notebook:
1. Loads the trained SAE model
2. Attaches it to the language model
3. Enables text tracking
4. Runs inference to collect top texts for each neuron
5. Exports the top texts to JSON

In [6]:
%load_ext autoreload
%autoreload 2

import torch
from pathlib import Path

from mi_crow.datasets import TextDataset
from mi_crow.language_model.language_model import LanguageModel
from mi_crow.mechanistic.sae.modules.topk_sae import TopKSae
from mi_crow.store.local_store import LocalStore

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
MODEL_ID = "speakleash/Bielik-1.5B-v3.0-Instruct"
HF_DATASET = "roneneldan/TinyStories"
DATA_SPLIT = "train"
TEXT_FIELD = "text"
DATA_LIMIT = 32
MAX_LENGTH = 128
BATCH_SIZE = 16

STORE_DIR = Path("./store")
TOP_TEXTS_FILE = STORE_DIR / "runs" / "training_20251213_005514" / "top_texts.json"
SAE_MODEL_PATH = STORE_DIR / "runs" / "training_20251213_005514" / "model.pt"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"📱 Using device: {DEVICE}")
print(f"🔧 Model: {MODEL_ID}")
print(f"📊 Dataset: {HF_DATASET}")
print(f"💾 SAE model: {SAE_MODEL_PATH}")
print(f"💾 Output file: {TOP_TEXTS_FILE}")

📱 Using device: cpu
🔧 Model: speakleash/Bielik-1.5B-v3.0-Instruct
📊 Dataset: roneneldan/TinyStories
💾 SAE model: store/runs/training_20251213_005514/model.pt
💾 Output file: store/runs/training_20251213_005514/top_texts.json


In [8]:
print("📥 Loading language model...")
store = LocalStore(base_path=STORE_DIR)
lm = LanguageModel.from_huggingface(MODEL_ID, store=store)
lm.model.to(DEVICE)

print(f"✅ Model loaded: {lm.model_id}")

📥 Loading language model...
✅ Model loaded: speakleash_Bielik-1.5B-v3.0-Instruct


In [9]:
run_id_file = STORE_DIR / "run_id.txt"
with open(run_id_file, 'r') as f:
    RUN_ID = f.read().strip()

meta = store.get_run_metadata(RUN_ID)
layer_signatures = meta.get('layer_signatures', [])
layer_signature = layer_signatures[0] if layer_signatures else None

print(f"📁 Run ID: {RUN_ID}")
print(f"🎯 Layer: {layer_signature}")

📁 Run ID: activations_20251212_172220
🎯 Layer: llamaforcausallm_model_layers_16_post_attention_layernorm


In [10]:
print("📥 Loading SAE model...")
sae = TopKSae.load(SAE_MODEL_PATH)
sae.context.device = DEVICE
sae.sae_engine.to(DEVICE)

if layer_signature is None and sae.context.lm_layer_signature is not None:
    layer_signature = sae.context.lm_layer_signature
    print(f"📝 Using layer signature from SAE context: {layer_signature}")

print(f"✅ SAE loaded")
print(f"   Input dimension: {sae.context.n_inputs}")
print(f"   Latent dimension: {sae.context.n_latents}")
print(f"   TopK: {sae.k}")
print(f"   Layer signature: {layer_signature}")

📥 Loading SAE model...


2025-12-13 23:13:51,046 [INFO] amber.mechanistic.sae.modules.topk_sae: 
Loaded TopKSAE from store/runs/training_20251213_005514/model.pt
n_latents=6144, n_inputs=1536, k=8


✅ SAE loaded
   Input dimension: 1536
   Latent dimension: 6144
   TopK: 8
   Layer signature: llamaforcausallm_model_layers_16_post_attention_layernorm


In [11]:
print("🔗 Attaching SAE to language model...")
if layer_signature is None:
    print("❌ Error: Layer signature is None. Cannot attach SAE.")
else:
    hook_id = lm.layers.register_hook(layer_signature, sae)
    print(f"✅ SAE attached to layer: {layer_signature}")
    print(f"   Hook ID: {hook_id}")

🔗 Attaching SAE to language model...
✅ SAE attached to layer: llamaforcausallm_model_layers_16_post_attention_layernorm
   Hook ID: 547adbd3-be2c-4ef3-a7c0-821c4c1a4ecb


In [12]:
print("📊 Enabling text tracking...")
sae.concepts.enable_text_tracking()
print("✅ Text tracking enabled")

📊 Enabling text tracking...
✅ Text tracking enabled


In [13]:
print("📥 Loading dataset...")
dataset = TextDataset.from_huggingface(
    HF_DATASET,
    split=DATA_SPLIT,
    store=store,
    text_field=TEXT_FIELD,
    limit=DATA_LIMIT,
)
print(f"✅ Loaded {len(dataset)} text samples")

📥 Loading dataset...


Saving the dataset (1/1 shards): 100%|██████████| 32/32 [00:00<00:00, 16155.24 examples/s]

✅ Loaded 32 text samples


In [14]:
print("🚀 Running inference to collect top texts...")
print(f"   Processing {len(dataset)} samples in batches of {BATCH_SIZE}")

with torch.inference_mode():
    for batch_idx, batch in enumerate(dataset.iter_batches(BATCH_SIZE)):
        texts = dataset.extract_texts_from_batch(batch)
        lm.forwards(
            texts,
            tok_kwargs={
                "max_length": MAX_LENGTH,
                "padding": True,
                "truncation": True,
                "add_special_tokens": True
            },
            autocast=False,
        )

        print(f"   Processed {batch_idx + 1} batches...")

print("✅ Inference completed")

🚀 Running inference to collect top texts...
   Processing 32 samples in batches of 16
   Processed 1 batches...
   Processed 2 batches...
✅ Inference completed


In [15]:
print("💾 Exporting top texts...")
exported_path = sae.concepts.export_top_texts_to_json(TOP_TEXTS_FILE)
print(f"✅ Top texts exported to: {exported_path}")

all_top_texts = sae.concepts.get_all_top_texts()
neurons_with_texts = sum(1 for texts in all_top_texts if texts)
total_texts = sum(len(texts) for texts in all_top_texts)

print(f"📊 Statistics:")
print(f"   Neurons with texts: {neurons_with_texts} / {sae.context.n_latents}")
print(f"   Total texts collected: {total_texts}")
print(f"   Average texts per neuron: {total_texts / sae.context.n_latents:.2f}")

💾 Exporting top texts...
✅ Top texts exported to: store/runs/training_20251213_005514/top_texts.json
📊 Statistics:
   Neurons with texts: 6144 / 6144
   Total texts collected: 30720
   Average texts per neuron: 5.00
